In [1]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import os
import glob
import json
import shutil


# Variables globales

In [2]:
download_dir = "../data_download"
os.makedirs(download_dir, exist_ok=True)

# Carpeta de salida para los Parquet
parquet_dir = "../data_parquet"
os.makedirs(parquet_dir, exist_ok=True)

fecha_inicio = '2020-01-01'

# fecha_fin es el dia anterior a hoy
#fecha_fin = datetime.today() - timedelta(days=1)
fecha_fin = datetime(2025, 10, 21)


# Descargamos los precios de la luz

In [3]:
current_date = datetime.fromisoformat(fecha_inicio)

while current_date <= fecha_fin:
    fecha_descarga = current_date.strftime('%Y-%m-%d')

    # Construimos patrón de búsqueda
    pattern = os.path.join(download_dir, f"{download_dir}/precios_{fecha_descarga}.json")
    
    # Verificamos si ya hay un fichero para esa fecha
    ya_descargado = glob.glob(pattern)
    if ya_descargado:
        print(f"Ya descargado: {fecha_descarga}, se omite.")
        current_date += timedelta(days=1)
        continue
    
    file = requests.get(f"https://api.esios.ree.es/archives/70/download?date={fecha_descarga}").content
    print(f"descargado {fecha_descarga}")

    with open(f"{download_dir}/precios_{fecha_descarga}.json", "wb") as f:
        f.write(file)
    
    current_date += timedelta(days=1)


Ya descargado: 2020-01-01, se omite.
Ya descargado: 2020-01-02, se omite.
Ya descargado: 2020-01-03, se omite.
Ya descargado: 2020-01-04, se omite.
Ya descargado: 2020-01-05, se omite.
Ya descargado: 2020-01-06, se omite.
Ya descargado: 2020-01-07, se omite.
Ya descargado: 2020-01-08, se omite.
Ya descargado: 2020-01-09, se omite.
Ya descargado: 2020-01-10, se omite.
Ya descargado: 2020-01-11, se omite.
Ya descargado: 2020-01-12, se omite.
Ya descargado: 2020-01-13, se omite.
Ya descargado: 2020-01-14, se omite.
Ya descargado: 2020-01-15, se omite.
Ya descargado: 2020-01-16, se omite.
Ya descargado: 2020-01-17, se omite.
Ya descargado: 2020-01-18, se omite.
Ya descargado: 2020-01-19, se omite.
Ya descargado: 2020-01-20, se omite.
Ya descargado: 2020-01-21, se omite.
Ya descargado: 2020-01-22, se omite.
Ya descargado: 2020-01-23, se omite.
Ya descargado: 2020-01-24, se omite.
Ya descargado: 2020-01-25, se omite.
Ya descargado: 2020-01-26, se omite.
Ya descargado: 2020-01-27, se omite.
Y

# Generamos tabla en parquet

In [4]:

current_date = datetime.fromisoformat(fecha_inicio)

df_list = []
while current_date <= fecha_fin:

    fecha_parquet = current_date.strftime('%Y-%m-%d')

    file_path = f"{download_dir}/precios_{fecha_parquet}.json"

    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Extraer la sección 'PVPC'
    pvpc_data = data['PVPC']
    df_dia = pd.DataFrame(pvpc_data)

    # Convertimos los valores numéricos que están en formato object a float
    df_dia = df_dia.map(lambda x: float(x.replace(',', '.')) if isinstance(x, str) and ',' in x else x)

    df_dia = df_dia.dropna(how='all')
    df_dia = df_dia.reset_index(drop=True)
    df_dia['year'] = str(current_date.year)
    df_dia['hour'] =df_dia["Hora"].str.split("-").str[0]
    

    if "GEN" in df_dia:
        df_dia["price"] = df_dia["GEN"]
    elif "PCB" in df_dia:
        df_dia["price"] = df_dia["PCB"]
    else:
        continue

    df_list.append(df_dia)

    current_date += timedelta(days=1)

df_precios = pd.concat(df_list)

ruta_precios = f"{parquet_dir}/precios_luz"
if os.path.exists(ruta_precios):
    shutil.rmtree(ruta_precios)
    
df_precios.to_parquet(ruta_precios, index=False, compression='snappy', partition_cols=['year'])

## Guardado como parquet, ahora cargamos el dataset

In [5]:

df = pd.read_parquet(f"{parquet_dir}/precios_luz")
df

,Dia,Hora,GEN,NOC,VHC,COFGEN,COFNOC,COFVHC,PMHGEN,PMHNOC,...,TEUCYM,CCVPCB,CCVCYM,EDSRPCB,EDSRCYM,EDCGASPCB,EDCGASCYM,TAHPCB,TAHCYM,year
0,01/01/2020,00-01,107.12,57.59,61.80,0.000108,0.000178,0.000149,52.07,49.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
1,01/01/2020,01-02,104.07,54.69,51.42,0.000091,0.000163,0.000164,48.17,45.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
2,01/01/2020,02-03,103.72,54.12,50.80,0.000076,0.000148,0.000160,46.36,43.97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
3,01/01/2020,03-04,100.27,50.84,47.64,0.000067,0.000136,0.000153,41.34,39.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
4,01/01/2020,04-05,98.81,49.46,46.31,0.000062,0.000130,0.000145,39.50,37.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50898,21/10/2025,19-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92.54,4.48,4.48,0.0,0.0,NaN,NaN,20.67,20.67,2025
50899,21/10/2025,20-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92.54,4.81,4.81,0.0,0.0,NaN,NaN,20.68,20.68,2025
50900,21/10/2025,21-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92.54,4.51,4.51,0.0,0.0,NaN,NaN,20.76,20.76,2025
50901,21/10/2025,22-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92.54,4.55,4.59,0.0,0.0,NaN,NaN,21.25,21.34,2025


In [6]:
df[df['Hora'].isna()]


,Dia,Hora,GEN,NOC,VHC,COFGEN,COFNOC,COFVHC,PMHGEN,PMHNOC,...,TEUCYM,CCVPCB,CCVCYM,EDSRPCB,EDSRCYM,EDCGASPCB,EDCGASCYM,TAHPCB,TAHCYM,year


In [7]:
df.describe()

,GEN,NOC,VHC,COFGEN,COFNOC,COFVHC,PMHGEN,PMHNOC,PMHVHC,SAHGEN,...,TEUPCB,TEUCYM,CCVPCB,CCVCYM,EDSRPCB,EDSRCYM,EDCGASPCB,EDCGASCYM,TAHPCB,TAHCYM
count,12407.000000,12407.000000,12407.000000,12407.000000,12407.000000,12407.000000,12407.000000,12407.000000,12407.000000,12407.000000,...,38496.000000,38496.000000,38496.000000,38496.000000,38496.0,38496.0,24816.000000,24816.000000,15839.000000,15839.000000
mean,101.846556,81.285609,81.644931,0.000110,0.000108,0.000110,46.958421,46.068922,46.439483,3.356671,...,27.177543,27.177543,3.745111,3.745183,0.0,0.0,16.954005,16.954309,5.673330,5.673449
std,21.964196,39.842079,40.119666,0.000036,0.000032,0.000034,22.277968,21.931713,22.265193,2.251167,...,32.841503,32.841503,1.220201,1.220266,0.0,0.0,48.869374,48.870408,13.324362,13.324709
min,55.430000,9.660000,8.530000,0.000047,0.000045,0.000052,0.010000,0.010000,0.010000,-0.230000,...,0.920000,0.920000,1.100000,1.100000,0.0,0.0,-18.810000,-18.810000,-28.700000,-28.700000
25%,87.430000,46.385000,46.270000,0.000077,0.000083,0.000077,32.475000,31.800000,31.870000,1.780000,...,2.990000,2.990000,2.930000,2.930000,0.0,0.0,0.000000,0.000000,-4.190000,-4.190000
50%,98.480000,75.570000,76.330000,0.000114,0.000100,0.000103,44.620000,43.730000,43.960000,2.680000,...,4.350000,4.350000,3.580000,3.580000,0.0,0.0,0.000000,0.000000,6.180000,6.180000
75%,111.545000,114.640000,115.130000,0.000136,0.000126,0.000141,57.565000,56.700000,57.480000,4.260000,...,31.510000,31.510000,4.390000,4.390000,0.0,0.0,0.000000,0.000000,16.400000,16.400000
max,202.720000,222.330000,222.330000,0.000214,0.000225,0.000181,145.870000,147.210000,147.210000,18.700000,...,133.120000,133.120000,13.740000,13.740000,0.0,0.0,468.900000,468.900000,33.560000,33.560000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50903 entries, 0 to 50902
Data columns (total 60 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Dia        50903 non-null  object  
 1   Hora       50903 non-null  object  
 2   GEN        12407 non-null  float64 
 3   NOC        12407 non-null  float64 
 4   VHC        12407 non-null  float64 
 5   COFGEN     12407 non-null  float64 
 6   COFNOC     12407 non-null  float64 
 7   COFVHC     12407 non-null  float64 
 8   PMHGEN     12407 non-null  float64 
 9   PMHNOC     12407 non-null  float64 
 10  PMHVHC     12407 non-null  float64 
 11  SAHGEN     12407 non-null  float64 
 12  SAHNOC     12407 non-null  float64 
 13  SAHVHC     12407 non-null  float64 
 14  FOMGEN     12407 non-null  float64 
 15  FOMNOC     12407 non-null  float64 
 16  FOMVHC     12407 non-null  float64 
 17  FOSGEN     12407 non-null  float64 
 18  FOSNOC     12407 non-null  float64 
 19  FOSVHC     12407 non-null